## GeoTIFF to netCDF

---
Convert CHIRPS GeoTIFF in a folder to single NetCDF file with time dimension enabled that is CF-Compliant
http://cfconventions.org/cf-conventions/v1.6.0/cf-conventions.html
 
Based on Rich Signell's answer on StackExchange: https://gis.stackexchange.com/a/70487
This script was tested using CHIRPS dekad data. Adjustment is needed if using other timesteps data for CHIRPS
NCO (http://nco.sourceforge.net) must be installed before using this script
 
Modified by
Benny Istanto, UN World Food Programme, benny.istanto@wfp.org

---

### CHIRPS Indonesia Dekad3

In [1]:
#!/usr/bin/env python
import numpy as np
import datetime as dt
import os
import gdal
import netCDF4
import re

In [2]:
ds = gdal.Open('/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/D3/idn_cli_chirps-v2.0.1981.01.3.tif')

In [3]:
a = ds.ReadAsArray()
nlat,nlon = np.shape(a)

In [4]:
b = ds.GetGeoTransform() #bbox, interval
lon = np.arange(nlon)*b[1]+b[0]
lat = np.arange(nlat)*b[5]+b[3]

basedate = dt.datetime(1980,1,1,0,0,0)

In [5]:
# create NetCDF file
nco = netCDF4.Dataset('idn_cli_chirps_1months_1981_2020_d3.nc','w',clobber=True)

# create dimensions, variables and attributes:
nco.createDimension('lon',nlon)
nco.createDimension('lat',nlat)
nco.createDimension('time',None)

timeo = nco.createVariable('time','f4',('time'))
timeo.units = 'days since 1980-1-1 00:00:00'
timeo.standard_name = 'time'
timeo.calendar = 'gregorian'
timeo.axis = 'T'

lono = nco.createVariable('lon','f4',('lon'))
lono.units = 'degrees_east'
lono.standard_name = 'longitude'
lono.long_name = 'longitude'
lono.axis = 'X'

lato = nco.createVariable('lat','f4',('lat'))
lato.units = 'degrees_north'
lato.standard_name = 'latitude'
lato.long_name = 'latitude'
lato.axis = 'Y'

# create container variable for CRS: lon/lat WGS84 datum
crso = nco.createVariable('crs','i4')
crso.long_name = 'Lon/Lat Coords in WGS84'
crso.grid_mapping_name='latitude_longitude'
crso.longitude_of_prime_meridian = 0.0
crso.semi_major_axis = 6378137.0
crso.inverse_flattening = 298.257223563

# create short integer variable for precipitation data, with chunking
pcpo = nco.createVariable('precip', 'f4',  ('time', 'lat', 'lon'),zlib=True,fill_value=-9999.)
pcpo.units = 'mm'
pcpo.standard_name = 'convective precipitation rate'
pcpo.long_name = 'Climate Hazards group InfraRed Precipitation with Stations'
pcpo.time_step = 'month'
pcpo.missing_value = -9999.
pcpo.geospatial_lat_min = -11.05
pcpo.geospatial_lat_max = 5.90
pcpo.geospatial_lon_min = 95.00
pcpo.geospatial_lon_max = 141.00
pcpo.grid_mapping = 'crs'
pcpo.set_auto_maskandscale(False)

nco.Conventions='CF-1.6'
nco.title = "CHIRPS v2.0"
nco.history = "created by Climate Hazards Group. University of California at Santa Barbara"
nco.version = "Version 2.0"
nco.comments = "time variable denotes the first day of the given dekad."
nco.website = "https://www.chc.ucsb.edu/data/chirps"
nco.date_created = "2021-01-08"
nco.creator_name = "Benny Istanto"
nco.creator_email = "benny.istanto@wfp.org"
nco.institution = "UN World Food Programme"
nco.note = "The data is developed to support regular updating procedure for SPI analysis (https://github.com/wfpidn/SPI). This activities will support WFP to assess extreme dry and wet periods as part of WFP's Seasonal Monitoring"


#write lon,lat
lono[:]=lon
lato[:]=lat

pat = re.compile('idn_cli_chirps-v2.0.[0-9]{4}\.[0-9]{2}\.[0-9]{1}')
itime=0

#step through data, writing time and data to NetCDF
for root, dirs, files in os.walk('/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/D3/'):
    dirs.sort()
    files.sort()
    for f in files:
        if re.match(pat,f):
            # read the time values by parsing the filename
            year=int(f[20:24])
            mon=int(f[25:27])
            dekad=int(f[28:29])
            date=dt.datetime(year,mon,dekad,0,0,0)
            print(date)
            dtime=(date-basedate).total_seconds()/86400.
            timeo[itime]=dtime
           # precipitation
            pcp_path = os.path.join(root,f)
            print(pcp_path)
            pcp=gdal.Open(pcp_path)
            a=pcp.ReadAsArray()  #data
            pcpo[itime,:,:]=a
            itime=itime+1

nco.close()

1981-01-03 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/D3/idn_cli_chirps-v2.0.1981.01.3.tif
1981-02-03 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/D3/idn_cli_chirps-v2.0.1981.02.3.tif
1981-03-03 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/D3/idn_cli_chirps-v2.0.1981.03.3.tif
1981-04-03 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/D3/idn_cli_chirps-v2.0.1981.04.3.tif
1981-05-03 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/D3/idn_cli_chirps-v2.0.1981.05.3.tif
1981-06-03 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/D3/idn_cli_chirps-v2.0.1981.06.3.tif
1981-07-03 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/D3/idn_cli_chirps-v2.0.1981.07.3.tif
1981-08-03 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/D3/idn_cli_chirps-v2.0.1981.08.3.tif
1981-09-03 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/D3/idn_cli_chirps-v2.0.1981.09.3.tif
1981-10-03 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/D3/idn_cli_chirps-v2.0.1981.10.3.tif
1981-11-03 00:00:00
/Users/ben

1988-11-03 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/D3/idn_cli_chirps-v2.0.1988.11.3.tif
1988-12-03 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/D3/idn_cli_chirps-v2.0.1988.12.3.tif
1989-01-03 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/D3/idn_cli_chirps-v2.0.1989.01.3.tif
1989-02-03 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/D3/idn_cli_chirps-v2.0.1989.02.3.tif
1989-03-03 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/D3/idn_cli_chirps-v2.0.1989.03.3.tif
1989-04-03 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/D3/idn_cli_chirps-v2.0.1989.04.3.tif
1989-05-03 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/D3/idn_cli_chirps-v2.0.1989.05.3.tif
1989-06-03 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/D3/idn_cli_chirps-v2.0.1989.06.3.tif
1989-07-03 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/D3/idn_cli_chirps-v2.0.1989.07.3.tif
1989-08-03 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/D3/idn_cli_chirps-v2.0.1989.08.3.tif
1989-09-03 00:00:00
/Users/ben

1996-12-03 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/D3/idn_cli_chirps-v2.0.1996.12.3.tif
1997-01-03 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/D3/idn_cli_chirps-v2.0.1997.01.3.tif
1997-02-03 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/D3/idn_cli_chirps-v2.0.1997.02.3.tif
1997-03-03 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/D3/idn_cli_chirps-v2.0.1997.03.3.tif
1997-04-03 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/D3/idn_cli_chirps-v2.0.1997.04.3.tif
1997-05-03 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/D3/idn_cli_chirps-v2.0.1997.05.3.tif
1997-06-03 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/D3/idn_cli_chirps-v2.0.1997.06.3.tif
1997-07-03 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/D3/idn_cli_chirps-v2.0.1997.07.3.tif
1997-08-03 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/D3/idn_cli_chirps-v2.0.1997.08.3.tif
1997-09-03 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/D3/idn_cli_chirps-v2.0.1997.09.3.tif
1997-10-03 00:00:00
/Users/ben

2004-03-03 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/D3/idn_cli_chirps-v2.0.2004.03.3.tif
2004-04-03 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/D3/idn_cli_chirps-v2.0.2004.04.3.tif
2004-05-03 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/D3/idn_cli_chirps-v2.0.2004.05.3.tif
2004-06-03 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/D3/idn_cli_chirps-v2.0.2004.06.3.tif
2004-07-03 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/D3/idn_cli_chirps-v2.0.2004.07.3.tif
2004-08-03 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/D3/idn_cli_chirps-v2.0.2004.08.3.tif
2004-09-03 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/D3/idn_cli_chirps-v2.0.2004.09.3.tif
2004-10-03 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/D3/idn_cli_chirps-v2.0.2004.10.3.tif
2004-11-03 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/D3/idn_cli_chirps-v2.0.2004.11.3.tif
2004-12-03 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/D3/idn_cli_chirps-v2.0.2004.12.3.tif
2005-01-03 00:00:00
/Users/ben

2011-04-03 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/D3/idn_cli_chirps-v2.0.2011.04.3.tif
2011-05-03 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/D3/idn_cli_chirps-v2.0.2011.05.3.tif
2011-06-03 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/D3/idn_cli_chirps-v2.0.2011.06.3.tif
2011-07-03 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/D3/idn_cli_chirps-v2.0.2011.07.3.tif
2011-08-03 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/D3/idn_cli_chirps-v2.0.2011.08.3.tif
2011-09-03 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/D3/idn_cli_chirps-v2.0.2011.09.3.tif
2011-10-03 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/D3/idn_cli_chirps-v2.0.2011.10.3.tif
2011-11-03 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/D3/idn_cli_chirps-v2.0.2011.11.3.tif
2011-12-03 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/D3/idn_cli_chirps-v2.0.2011.12.3.tif
2012-01-03 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/D3/idn_cli_chirps-v2.0.2012.01.3.tif
2012-02-03 00:00:00
/Users/ben

2018-09-03 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/D3/idn_cli_chirps-v2.0.2018.09.3.tif
2018-10-03 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/D3/idn_cli_chirps-v2.0.2018.10.3.tif
2018-11-03 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/D3/idn_cli_chirps-v2.0.2018.11.3.tif
2018-12-03 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/D3/idn_cli_chirps-v2.0.2018.12.3.tif
2019-01-03 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/D3/idn_cli_chirps-v2.0.2019.01.3.tif
2019-02-03 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/D3/idn_cli_chirps-v2.0.2019.02.3.tif
2019-03-03 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/D3/idn_cli_chirps-v2.0.2019.03.3.tif
2019-04-03 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/D3/idn_cli_chirps-v2.0.2019.04.3.tif
2019-05-03 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/D3/idn_cli_chirps-v2.0.2019.05.3.tif
2019-06-03 00:00:00
/Users/bennyistanto/Temp/CHIRPS/SPI/IDN/D3/idn_cli_chirps-v2.0.2019.06.3.tif
2019-07-03 00:00:00
/Users/ben